In [2]:
import cvxpy as cp
import numpy as np

# given parameters
a = 1.4  # x1 \in [-a a]
b = 0.7  # x3 \in [-b b]
Premise_var_x1 = np.array([[0, a/2, a]]) # set only positive values including 0
Premise_var_x3 = np.array([[0, b]]) # you can choose negative or positive values

n_r = np.size(Premise_var_x1)*np.size(Premise_var_x3)  # number of rules
n_s = 4  # size of A[i] matrix
n_u = 1  # size of B[i] matrix

# Define A and B matrices
A = []
B = []

A = []
for i in range(np.size(Premise_var_x1)):
  for j in range(np.size(Premise_var_x3)):
    x1 = Premise_var_x1[0,i]
    x3 = Premise_var_x3[0,j]
    if x3 == 0:
      A13 = 1
      A31 = x1**2
      A43 = 1
      B11 = x1**2
      A.append(np.array([[1, 1, A13, -0.1],
                      [1, -2, 0, 0],
                      [1, A31, -0.3, 0],
                      [0, 0, A43, -1]]))
    elif x3 == b:
      A13 = np.sin(x3)/x3
      A31 = x1**2
      A43 = np.sin(x3)/x3
      B11 = x1**2
      A.append(np.array([[1, 1, A13, -0.1],
                [1, -2, 0, 0],
                [1, A31, -0.3, 0],
                [0, 0, A43, -1]]))
    B.append(np.array([[1+B11],
                [0],
                [0],
                [0]]))
# decision variables
Q = cp.Variable((n_s, n_s), symmetric=True)
Y = [cp.Variable((n_u, n_s)) for _ in range(n_r)]
I = np.identity(n_s)
#gamma = cp.Variable((1, 1))
# LMI constraints
constraints = []

# LMI condition for stabilization of each sub system
for i in range(n_r):
    Ai = A[i]
    Bi = B[i]
    constraints.append((Ai @ Q + Q @ Ai.T + Bi @ Y[i] + Y[i].T @ Bi.T) << 0)

# LMI condition for stabilization of joint systems (refer [1])
# case 1
for j in range(1, n_r):
    Ai = A[0]
    Aj = A[j]
    Bi = B[0]
    Bj = B[j]
    constraint_lhs = (Ai @ Q + Q @ Ai.T + Bi @ Y[j] + Y[j].T @ Bi.T) / 2
    constraint_rhs = (Aj @ Q + Q @ Aj.T + Bj @ Y[0] + Y[0].T @ Bj.T) / 2
    constraints.append(constraint_lhs + constraint_rhs << 0)

# case 2
for j in range(2, n_r):
    Ai = A[1]
    Aj = A[j]
    Bi = B[1]
    Bj = B[j]
    constraint_lhs = (Ai @ Q + Q @ Ai.T + Bi @ Y[j] + Y[j].T @ Bi.T) / 2
    constraint_rhs = (Aj @ Q + Q @ Aj.T + Bj @ Y[1] + Y[1].T @ Bj.T) / 2
    constraints.append(constraint_lhs + constraint_rhs << 0)

# case 3
for j in range(3, n_r):
    Ai = A[2]
    Aj = A[j]
    Bi = B[2]
    Bj = B[j]
    constraint_lhs = (Ai @ Q + Q @ Ai.T + Bi @ Y[j] + Y[j].T @ Bi.T) / 2
    constraint_rhs = (Aj @ Q + Q @ Aj.T + Bj @ Y[2] + Y[2].T @ Bj.T) / 2
    constraints.append(constraint_lhs + constraint_rhs << 0)

# Positive semidefinite constraint on Q
constraints.append(Q - 0.1*I>> 0) # 0.1 is for avoiding numerical problems

problem = cp.Problem(cp.Minimize(0), constraints)
print(problem.is_qp())
problem.solve(verbose = True)  # you can choose SDP solvers (recommend "SCS" solver)


ch = [c.dual_value for c in constraints]

K = [Y[i].value @ np.linalg.inv(Q.value) for i in range(n_r)]

for i in range(n_r):
  print(f'K{{{i+1}}}=', K[i])

True
                                     CVXPY                                     
                                     v1.5.2                                    
(CVXPY) Aug 15 05:12:13 AM: Your problem has 40 variables, 304 constraints, and 0 parameters.
(CVXPY) Aug 15 05:12:13 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Aug 15 05:12:13 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Aug 15 05:12:13 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Aug 15 05:12:13 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Aug 15 05:12:13 AM: Compiling problem (target solver=SCS)